<div style="background-color: #4472C4; color: white; padding: 20px; border-radius: 10px;">
<h1 style="text-align: center; color: white;">📈 Option Analysis and Parameter Estimation Tutorial</h1>
<p style="text-align: center; font-style: italic; font-size: 16px;">A step-by-step guide to estimate rough volatility model parameters from options data</p>
</div>

<div style="background-color: #E6F0FF; padding: 15px; border-radius: 5px; margin-top: 20px;">
This notebook walks through a structured process for analyzing financial options data, specifically focused on estimating parameters for a rough volatility model. We'll learn how to:

<ol style="color: #333">
  <li><b>Load and preprocess</b> financial data</li>
  <li><b>Filter options</b> by date, ticker, and maturity</li>
  <li><b>Extract key pricing inputs</b> for option valuation</li>
  <li><b>Estimate model parameters</b> including correlation (ρ), roughness (H), and volatility of volatility (η)</li>
  <li><b>Set up global parameters</b> for option pricing models</li>
</ol>

<p style="text-align: center; margin-top: 15px;">Let's dive in! 🚀</p>
</div>

## 1. Importing Required Libraries
<div style="border-left: 5px solid #4472C4; padding-left: 10px;">
First, we'll import the necessary Python libraries for data handling, numerical computations, and file operations.
</div>

In [2]:
import numpy as np
import pandas as pd
import os
import logging
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set plot styling for better visualization
plt.style.use('ggplot')
sns.set_palette('Blues_r')
sns.set_context('notebook', font_scale=1.2)

# Set up logging to track our progress
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

## 2. Initializing Global Parameters
<div style="background-color: #F2F9FF; padding: 15px; border-radius: 5px; border-left: 5px solid #5B9BD5;">
Here we set up placeholders for our model parameters. These will be populated later in the notebook based on our analysis.

<table style="width:100%; border-collapse: collapse; margin-top:10px;">
  <tr style="background-color: #5B9BD5; color: white;">
    <th style="padding: 8px; text-align: left;">Parameter</th>
    <th style="padding: 8px; text-align: left;">Description</th>
  </tr>
  <tr style="background-color: #EAF2FA;">
    <td style="padding: 8px; font-family: monospace;">N1, N, T</td>
    <td style="padding: 8px;">Time discretization parameters</td>
  </tr>
  <tr>
    <td style="padding: 8px; font-family: monospace;">M, M2</td>
    <td style="padding: 8px;">Monte Carlo simulation parameters</td>
  </tr>
  <tr style="background-color: #EAF2FA;">
    <td style="padding: 8px; font-family: monospace;">eta</td>
    <td style="padding: 8px;">Volatility of volatility parameter</td>
  </tr>
  <tr>
    <td style="padding: 8px; font-family: monospace;">X0</td>
    <td style="padding: 8px;">Initial asset price (forward price)</td>
  </tr>
  <tr style="background-color: #EAF2FA;">
    <td style="padding: 8px; font-family: monospace;">r</td>
    <td style="padding: 8px;">Risk-free interest rate</td>
  </tr>
  <tr>
    <td style="padding: 8px; font-family: monospace;">rho</td>
    <td style="padding: 8px;">Correlation between asset returns and volatility</td>
  </tr>
  <tr style="background-color: #EAF2FA;">
    <td style="padding: 8px; font-family: monospace;">xi</td>
    <td style="padding: 8px;">Initial variance value</td>
  </tr>
  <tr>
    <td style="padding: 8px; font-family: monospace;">strike</td>
    <td style="padding: 8px;">Option strike price</td>
  </tr>
  <tr style="background-color: #EAF2FA;">
    <td style="padding: 8px; font-family: monospace;">K</td>
    <td style="padding: 8px;">Model calibration parameter</td>
  </tr>
</table>
</div>

In [3]:
# Global option model parameters (pre-initialized)
N1 = 0      # Time discretization for simulation
N = 0       # Time discretization reference
T = 0       # Time to maturity in days
T_years = 0.0  # Time to maturity in years
M = 0       # Number of Monte Carlo paths
M2 = 0      # Another Monte Carlo parameter
eta = 0.0   # Volatility of volatility
X0 = 0.0    # Initial price (forward price)
r = 0.0     # Risk-free rate
rho = 0.0   # Correlation
xi = 0.0    # Initial variance
strike = 0.0 # Strike price
K = 0       # Model parameter

### 2.1 The Payoff Function
<div style="background-color: #F8F8F8; padding: 12px; border-left: 4px solid #70AD47; margin: 10px 0;">
For put options, the payoff function determines the final value of the option at expiration. It returns the maximum of (strike price - asset price) or zero.

<div style="text-align: center; margin-top: 15px;">
$\text{Payoff} = \max(K - S_T, 0)$
</div>

<p style="text-align: center; font-style: italic; margin-top: 5px; color: #555;">where $K$ is the strike price and $S_T$ is the asset price at expiration</p>
</div>

In [4]:
def phi(x):
    """Calculate the put option payoff: max(strike - asset price, 0)"""
    return np.maximum(strike - x, 0)

## 3. Setting Up File Paths
<div style="background-color: #FFF4E1; padding: 10px; border-radius: 5px; border-left: 5px solid #FF9800;">
Now we define the file paths for our data. We use Path from the pathlib library to ensure cross-platform compatibility.
</div>

In [5]:
# Paths
repo_root = os.path.abspath('..')
notebook_dir = Path(repo_root) / 'XGboost_Roshan' / 'src'
DATA_PICKLE = notebook_dir / 'data' / 'dataset1.pkl'  # Options data
RETURNS_PICKLE = notebook_dir / 'data' / 'dataset3.pkl'  # Stock returns data

print(f"Data files will be loaded from:\n{DATA_PICKLE}\n{RETURNS_PICKLE}")

Data files will be loaded from:
/Users/roshanshah1/XGboost_Roshan/src/data/dataset1.pkl
/Users/roshanshah1/XGboost_Roshan/src/data/dataset3.pkl


## 4. Data Loading and Cleaning
<div style="background-color: #E8F4F9; padding: 15px; border-radius: 8px; border-left: 5px solid #00ACC1;">
This function loads data from pickle files and performs basic cleaning operations. If a pickle file is not found, it attempts to convert a CSV file with the same name.

<p style="margin-top: 10px;"><i class="fa fa-info-circle" style="color: #00ACC1;"></i> The function handles both file formats and ensures proper date formatting and column cleanup.</p>
</div>

In [6]:
def load_and_clean_data(pickle_path):
    """Load data from pickle file or convert from CSV if needed"""
    if os.path.exists(pickle_path):
        logger.info(f"Loading data from {pickle_path}")
        df = pd.read_pickle(pickle_path)
    else:
        logger.warning(f"Pickle file not found at {pickle_path}")
        os.makedirs(pickle_path.parent, exist_ok=True)
        csv_path = pickle_path.with_suffix('.csv')
        if os.path.exists(csv_path):
            logger.info(f"Found CSV at {csv_path}. Converting to pickle.")
            df = pd.read_csv(csv_path, parse_dates=['date'], dayfirst=False)
            df.to_pickle(pickle_path)
            logger.info(f"Converted and saved as pickle: {pickle_path}")
        else:
            logger.warning(f"CSV file also not found at {csv_path}")
            return None

    # Clean up the dataframe
    if 'secid' in df.columns or 'index_flag' in df.columns:
        df = df.drop(columns=['secid', 'index_flag'], errors='ignore')
    if 'date' in df.columns:
        df['date'] = pd.to_datetime(df['date'])

    logger.info(f"Loaded {df.shape[0]} rows and {df.shape[1]} columns from {pickle_path.name}")
    return df

# Load the datasets
df = load_and_clean_data(DATA_PICKLE)  # Options data
returns_df = load_and_clean_data(RETURNS_PICKLE)  # Returns data

# Display sample data
if df is not None and returns_df is not None:
    print("\nOptions Data Sample:")
    display(df.head(3))
    
    print("\nReturns Data Sample:")
    display(returns_df.head(3))
else:
    print("Error: Could not load one or both datasets")

2025-05-17 10:51:44,654 - WARNING - Pickle file not found at /Users/roshanshah1/XGboost_Roshan/src/data/dataset1.pkl
2025-05-17 10:51:44,655 - WARNING - CSV file also not found at /Users/roshanshah1/XGboost_Roshan/src/data/dataset1.csv
2025-05-17 10:51:44,655 - WARNING - Pickle file not found at /Users/roshanshah1/XGboost_Roshan/src/data/dataset3.pkl
2025-05-17 10:51:44,656 - WARNING - CSV file also not found at /Users/roshanshah1/XGboost_Roshan/src/data/dataset3.csv
2025-05-17 10:51:44,655 - WARNING - CSV file also not found at /Users/roshanshah1/XGboost_Roshan/src/data/dataset1.csv
2025-05-17 10:51:44,655 - WARNING - Pickle file not found at /Users/roshanshah1/XGboost_Roshan/src/data/dataset3.pkl
2025-05-17 10:51:44,656 - WARNING - CSV file also not found at /Users/roshanshah1/XGboost_Roshan/src/data/dataset3.csv


Error: Could not load one or both datasets


## 5. Data Filtering Functions
<div style="background-color: #F5F5F5; padding: 15px; border-radius: 5px; border-left: 5px solid #9C27B0;">
Now let's define functions to filter our data by date, option maturity (days), and to extract model inputs. These functions help us efficiently select the precise data we need for our analysis.

<p style="margin-top: 10px;"><span style="color: #9C27B0; font-weight: bold;">❓ Why filter by days to maturity?</span> Options with different maturities have different pricing characteristics. Keeping the maturity constant allows us to isolate other factors.</p>
</div>

In [7]:
def select_data_by_date_and_days(df, target_date=None, days_length=None):
    """Filter option data by date and days to maturity"""
    filtered_df = df.copy()
    if target_date:
        filtered_df = filtered_df[filtered_df['date'] == pd.to_datetime(target_date)]
    if days_length is not None:
        filtered_df = filtered_df[filtered_df['days'] == days_length]
    return filtered_df

def extract_model_inputs(option_df):
    """Extract key model inputs from put option data"""
    put_df = option_df[option_df['cp_flag'] == 'P']  # Filter for put options only
    if put_df.empty:
        return None
    
    # Take the first put option as our reference
    row = put_df.iloc[0]
    return {
        "ticker": row['ticker'],
        "date": row['date'],
        "T_years": row['days'] / 252,  # Convert days to years (252 trading days)
        "strike": row['strike_price'],
        "X0 (forward)": row['forward_price'],
        "market_premium": row['premium']  # Actual market price of the option
    }

## 6. Parameter Estimation Functions
<div style="background-color: #EFF8F7; padding: 15px; border-radius: 8px; border-left: 5px solid #26A69A;">
Now we'll define functions to estimate three critical model parameters for rough volatility models:

<div style="display: flex; justify-content: space-around; margin: 20px 0;">
  <div style="text-align: center; padding: 10px; width: 30%; background-color: #B2DFDB; border-radius: 8px;">
    <h3 style="margin: 0; color: #004D40;">ρ (rho)</h3>
    <p>Correlation between<br>returns and volatility</p>
  </div>
  <div style="text-align: center; padding: 10px; width: 30%; background-color: #B2DFDB; border-radius: 8px;">
    <h3 style="margin: 0; color: #004D40;">H (Hurst)</h3>
    <p>Roughness of the<br>volatility process</p>
  </div>
  <div style="text-align: center; padding: 10px; width: 30%; background-color: #B2DFDB; border-radius: 8px;">
    <h3 style="margin: 0; color: #004D40;">η (eta)</h3>
    <p>Volatility of<br>volatility</p>
  </div>
</div>

These parameters are crucial for capturing the dynamics of financial markets accurately.
</div>

### 6.1 Estimating Correlation (rho)
<div style="background-color: #E3F2FD; padding: 12px; border-radius: 5px; border-left: 4px solid #2196F3;">
This function estimates the correlation between stock returns and changes in implied volatility, which is a key parameter in stochastic volatility models.

<p style="margin-top: 10px;">A negative correlation (typically between -0.7 and -0.9 for equities) captures the <b>leverage effect</b>: when stock prices fall, volatility tends to rise.</p>
</div>

In [8]:
def estimate_rho(returns_df, options_df, target_ticker, target_date, days_length):
    """Estimate correlation between returns and volatility changes"""
    # Filter returns data
    ret_df = returns_df[(returns_df['ticker'] == target_ticker) & 
                         (returns_df['date'] <= target_date)].copy()
    
    # Filter options data for put options with specific maturity
    opt_df = options_df[(options_df['ticker'] == target_ticker) & 
                        (options_df['cp_flag'] == 'P') & 
                        (options_df['days'] == days_length) & 
                        (options_df['date'] <= target_date)].copy()

    # Sort chronologically
    ret_df = ret_df.sort_values('date')
    opt_df = opt_df.sort_values('date').drop_duplicates(subset=['date'])

    # Merge the datasets and calculate volatility changes
    merged = pd.merge(ret_df, opt_df[['date', 'impl_volatility']], on='date', how='inner')
    merged['vol_change'] = merged['impl_volatility'].diff()
    merged = merged.dropna(subset=['return', 'vol_change'])

    # Need at least 2 data points to calculate correlation
    if len(merged) < 2:
        return None
    
    # Return correlation coefficient between returns and volatility changes
    return merged['return'].corr(merged['vol_change'])

### 6.2 Computing Hurst Exponent
<div style="background-color: #F1F8E9; padding: 12px; border-radius: 5px; border-left: 4px solid #8BC34A;">
The Hurst exponent measures the long-term memory of a time series. It helps us understand how "rough" or "smooth" our volatility process is:

<div style="display: flex; flex-direction: row; margin: 15px 0;">
  <div style="flex: 1; text-align: center; padding: 10px; background-color: #DCEDC8; border-radius: 5px; margin-right: 5px;">
    <h4 style="margin-top: 0; color: #33691E;">H < 0.5</h4>
    <p>Rough/anti-persistent</p>
    <p><small>Series tends to reverse direction</small></p>
  </div>
  <div style="flex: 1; text-align: center; padding: 10px; background-color: #DCEDC8; border-radius: 5px; margin-right: 5px;">
    <h4 style="margin-top: 0; color: #33691E;">H = 0.5</h4>
    <p>Random walk</p>
    <p><small>No correlation between points</small></p>
  </div>
  <div style="flex: 1; text-align: center; padding: 10px; background-color: #DCEDC8; border-radius: 5px;">
    <h4 style="margin-top: 0; color: #33691E;">H > 0.5</h4>
    <p>Smooth/persistent</p>
    <p><small>Series tends to continue trend</small></p>
  </div>
</div>

<p>Our implementation uses the rescaled range (R/S) analysis method, a classical technique for estimating the Hurst exponent.</p>

<p style="font-style: italic; margin-top: 5px; color: #558B2F;">💡 Empirical studies show that volatility typically has H ≈ 0.1, making volatility a "rough" process.</p>
</div>

In [9]:
def compute_rolling_hurst(returns: np.ndarray, power: int) -> np.ndarray:
    """Compute rolling Hurst exponent using R/S analysis"""
    if not isinstance(returns, np.ndarray):
        returns = np.array(returns)
    
    # Window size
    n = 2**power
    if len(returns) < n:
        raise ValueError(f"Need at least {n} data points for power={power}")
    
    hursts = []
    exponents = np.arange(2, power+1)
    
    # Calculate Hurst exponent for each window
    for t in range(n, len(returns) + 1):
        window = returns[t-n:t]
        rs_log = []
        
        # For each scale
        for exp in exponents:
            m = 2**exp  # Length of segment
            s = n // m  # Number of segments
            
            # Reshape returns into segments
            segments = window.reshape(s, m)
            
            # Calculate cumulative deviations from mean
            dev = np.cumsum(segments - segments.mean(axis=1, keepdims=True), axis=1)
            
            # Range (R) = max - min of deviations
            R = dev.max(axis=1) - dev.min(axis=1)
            
            # Standard deviation (S) of each segment
            S = segments.std(axis=1)
            
            # R/S ratio (avoid division by zero)
            rs = np.where(S != 0, R/S, 0)
            
            # Log of average R/S for this scale
            rs_log.append(np.log2(rs.mean()))
        
        # Slope of log(R/S) vs log(scale) gives the Hurst exponent
        hursts.append(np.polyfit(exponents, rs_log, 1)[0])
    
    return np.array(hursts)

### 6.3 Estimating Volatility Parameters
<div style="background-color: #FBE9E7; padding: 12px; border-radius: 5px; border-left: 4px solid #FF5722;">
This function estimates two key volatility parameters:

<div style="display: flex; margin: 15px 0;">
  <div style="flex: 1; text-align: center; padding: 10px; background-color: #FFCCBC; border-radius: 5px; margin-right: 10px;">
    <h4 style="margin-top: 0; color: #BF360C;">η (eta)</h4>
    <p><b>Volatility of volatility</b></p>
    <p><small>Measures how much the volatility itself fluctuates</small></p>
  </div>
  <div style="flex: 1; text-align: center; padding: 10px; background-color: #FFCCBC; border-radius: 5px;">
    <h4 style="margin-top: 0; color: #BF360C;">ξ (xi)</h4>
    <p><b>Initial variance</b></p>
    <p><small>The starting point for the variance process, calculated as the square of current implied volatility</small></p>
  </div>
</div>

<p>The estimation uses log-volatility differences and scales them according to the Hurst exponent to capture the rough volatility dynamics.</p>
</div>

In [10]:
def estimate_eta_and_xi(option_df, hurst):
    """Estimate volatility model parameters"""
    # Sort and remove duplicates
    option_df = option_df.sort_values('date').drop_duplicates(subset=['date'])
    
    # Calculate log volatility and its differences
    option_df['log_vol'] = np.log(option_df['impl_volatility'])
    option_df['log_vol_diff'] = option_df['log_vol'].diff()
    
    # Standard deviation of log volatility differences
    std_log_vol_diff = option_df['log_vol_diff'].dropna().std()
    
    # Time step (1 trading day in years)
    delta_t = 1 / 252
    
    # Scale the standard deviation according to the Hurst exponent
    eta = std_log_vol_diff / (delta_t ** hurst)
    
    # Initial variance is the square of the first implied volatility
    xi = option_df['impl_volatility'].iloc[0] ** 2 if not option_df['impl_volatility'].isnull().all() else 0.0
    
    return eta, xi

## 7. Main Analysis Process
<div style="background-color: #E8EAF6; padding: 15px; border-radius: 8px; border-left: 5px solid #3F51B5;">
Now let's define our main analysis function that brings everything together. This function will:

<ol style="color: #333; margin-top: 10px;">
  <li>Filter the option data by ticker, date, and days to maturity</li>
  <li>Extract model inputs from the filtered put options</li>
  <li>Estimate the correlation (ρ) between returns and volatility</li>
  <li>Compute the Hurst exponent (H) for the roughness of volatility</li>
  <li>Estimate volatility parameters (η and ξ)</li>
  <li>Set up all required model configuration parameters</li>
</ol>

<p style="text-align: center; margin-top: 15px; font-style: italic; color: #3F51B5;">The analysis function brings together all our parameter estimation techniques into a cohesive workflow.</p>
</div>

In [11]:
def run_analysis(df, returns_df, ticker="AAPL", date_str="2023-08-31", days_length=10):
    """Run the full analysis for a given ticker, date, and maturity"""
    target_date = pd.to_datetime(date_str)
    
    # Step 1: Filter data by date and days to maturity
    filtered_data = select_data_by_date_and_days(df, target_date, days_length)
    ticker_data = filtered_data[filtered_data['ticker'] == ticker]
    
    if len(ticker_data) == 0:
        return f"No data found for {ticker} on {target_date.date()} with {days_length} days maturity"
    
    # Step 2: Extract model inputs from put options
    model_inputs = extract_model_inputs(ticker_data)
    if model_inputs is None:
        return f"No put options found for {ticker} on {target_date.date()} with {days_length} days maturity"
    
    # Step 3: Estimate correlation (rho)
    rho_est = estimate_rho(returns_df, df, ticker, target_date, days_length)
    rho = round(rho_est, 2) if rho_est is not None else -0.9  # Default to -0.9 if estimation fails
    
    # Step 4: Compute Hurst exponent
    rets = returns_df[(returns_df['ticker'] == ticker) & (returns_df['date'] <= target_date)]
    rets = rets.sort_values('date')['return'].values
    
    try:
        hurst_values = compute_rolling_hurst(rets, power=5)
        hurst_dates = returns_df[(returns_df['ticker'] == ticker) & 
                                 (returns_df['date'] <= target_date)].sort_values('date').iloc[2**5 - 1:]['date'].values
        hurst_df = pd.DataFrame({'date': hurst_dates, 'H': hurst_values})
        H_df = hurst_df[hurst_df['date'] <= target_date]
        latest_H = H_df.iloc[-1]['H'] if not H_df.empty else 0.1
    except Exception as e:
        print(f"Error in Hurst calculation: {e}")
        latest_H = 0.1  # Default value
    
    # Step 5: Estimate volatility parameters
    iv_path_df = df[(df['ticker'] == ticker) &
                    (df['cp_flag'] == 'P') &
                    (df['days'] == days_length) &
                    (df['date'] <= target_date)].sort_values('date')
    
    eta, xi = estimate_eta_and_xi(iv_path_df, hurst=latest_H)
    eta = round(eta, 3)
    xi = round(xi, 5)

    N = 252  # One year of trading days
    M = 2**8  # Number of Monte Carlo paths
    M2 = 2**8  # Another Monte Carlo parameter
    r = 0.05  # Risk-free rate (5%)
    K = 2     # Model parameter
    
    T = int(model_inputs["T_years"] * 252)  # Time to maturity in days
    N1 = T  # Time discretization
    T_years = model_inputs["T_years"]  # Time to maturity in years
    X0 = model_inputs["X0 (forward)"]  # Forward price
    strike = model_inputs["strike"]  # Strike price
    premium = model_inputs["market_premium"]  # Market premium
    
    # Create a results dictionary
    results = {
        "ticker": ticker,
        "date": target_date,
        "days_to_maturity": days_length,
        "forward_price": X0,
        "strike_price": strike,
        "market_premium": premium,
        "estimated_rho": rho,
        "hurst_exponent": round(latest_H, 3),
        "eta": eta,
        "xi": xi,
        "T": T,
        "T_years": round(T_years, 5),
        "N": N,
        "N1": N1,
        "M": M,
        "M2": M2,
        "r": r,
        "K": K
    }
    
    return results

## 8. Interactive Analysis
<div style="background-color: #E0F7FA; padding: 15px; border-radius: 8px; border-left: 5px solid #00BCD4;">
Now let's run our analysis with interactive inputs. You can change the ticker, date, and days to maturity to analyze different options.

<p style="margin-top: 10px;">For a fully interactive experience, uncomment the ipywidgets code to create sliders and input fields.</p>
</div>

In [12]:
# Default values
default_ticker = "AAPL"
default_date = "2023-08-31"
default_days = 10

# For interactive notebooks, uncomment these lines:
# from ipywidgets import interact, widgets

# @interact(
#    ticker=widgets.Text(value=default_ticker, description='Ticker:'),
#    date=widgets.Text(value=default_date, description='Date (YYYY-MM-DD):'),
#    days=widgets.IntSlider(min=5, max=90, step=5, value=default_days, description='Days to Maturity:')
# )

# Instead of widgets, use these variables directly
ticker = default_ticker
date = default_date
days = default_days

# Run the analysis
if df is not None and returns_df is not None:
    results = run_analysis(df, returns_df, ticker, date, days)
    
    if isinstance(results, dict):
        # Create a styled display for results
        from IPython.display import HTML, display
        import pandas as pd
        
        # Convert results to a styled DataFrame for better presentation
        market_data = pd.DataFrame({
            'Metric': ['Forward Price', 'Strike Price', 'Market Premium'],
            'Value': [f"${results['forward_price']:.2f}", 
                     f"${results['strike_price']:.2f}", 
                     f"${results['market_premium']:.2f}"]
        })
        
        parameters = pd.DataFrame({
            'Parameter': ['Correlation (ρ)', 'Hurst Exponent (H)', 'Volatility of Volatility (η)', 'Initial Variance (ξ)'],
            'Value': [f"{results['estimated_rho']:.3f}", 
                     f"{results['hurst_exponent']:.3f}", 
                     f"{results['eta']:.3f}", 
                     f"{results['xi']:.5f}"]
        })
        
        model_config = pd.DataFrame({
            'Setting': ['Time (days)', 'Time (years)', 'Monte Carlo Paths', 'Risk-free Rate'],
            'Value': [f"{results['T']}", 
                     f"{results['T_years']:.5f}", 
                     f"{results['M']}", 
                     f"{results['r']:.2f}"]
        })
        
        # Display header
        display(HTML(f"""<div style="background-color: #4472C4; color: white; padding: 10px; border-radius: 5px; text-align: center; margin-bottom: 15px;">
            <h3 style="margin: 0;">📊 Analysis Results for {results['ticker']} on {results['date'].date()} ({results['days_to_maturity']} days to maturity)</h3>
        </div>"""))
        
        # Display market data table with styling
        display(HTML("<h4 style='margin-bottom: 5px; color: #5B9BD5;'>Market Data</h4>"))
        display(market_data.style
                .set_properties(**{'text-align': 'left', 'border': '1px solid #dee2e6'})
                .set_table_styles([{'selector': 'th', 'props': [('background-color', '#5B9BD5'), 
                                                               ('color', 'white'),
                                                               ('text-align', 'left'),
                                                               ('padding', '8px')]}])
               )
        
        # Display estimated parameters table with styling
        display(HTML("<h4 style='margin-top: 20px; margin-bottom: 5px; color: #70AD47;'>Estimated Parameters</h4>"))
        display(parameters.style
                .set_properties(**{'text-align': 'left', 'border': '1px solid #dee2e6'})
                .set_table_styles([{'selector': 'th', 'props': [('background-color', '#70AD47'), 
                                                               ('color', 'white'),
                                                               ('text-align', 'left'),
                                                               ('padding', '8px')]}])
               )
        
        # Display model configuration table with styling
        display(HTML("<h4 style='margin-top: 20px; margin-bottom: 5px; color: #ED7D31;'>Model Configuration</h4>"))
        display(model_config.style
                .set_properties(**{'text-align': 'left', 'border': '1px solid #dee2e6'})
                .set_table_styles([{'selector': 'th', 'props': [('background-color', '#ED7D31'), 
                                                               ('color', 'white'),
                                                               ('text-align', 'left'),
                                                               ('padding', '8px')]}])
               )
        
        # Option Value Visualization
        plt.figure(figsize=(10, 6))
        
        # Range of stock prices to plot
        stock_prices = np.linspace(results['forward_price'] * 0.7, results['forward_price'] * 1.3, 100)
        
        # Calculate payoff at expiration
        payoffs = np.maximum(results['strike_price'] - stock_prices, 0)
        
        # Plot
        plt.plot(stock_prices, payoffs, 'r-', linewidth=2, label='Payoff at Expiration')
        plt.axvline(x=results['forward_price'], color='blue', linestyle='--', label='Current Forward Price')
        plt.axvline(x=results['strike_price'], color='green', linestyle='--', label='Strike Price')
        plt.axhline(y=results['market_premium'], color='purple', linestyle=':', label='Market Premium')
        
        plt.fill_between(stock_prices, payoffs, alpha=0.2, color='red')
        
        plt.title(f'Put Option Payoff Diagram for {results["ticker"]}', fontsize=14)
        plt.xlabel('Stock Price at Expiration', fontsize=12)
        plt.ylabel('Option Value/Payoff', fontsize=12)
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.tight_layout()
        plt.show()
        
    else:
        print(results)  # Print error message
else:
    print("Error: Data not available for analysis")

Error: Data not available for analysis


## 9. Understanding the Parameters
<div style="background-color: #FFEBEE; padding: 15px; border-radius: 8px; margin-bottom: 20px;">
Let's explain what each estimated parameter means and why they're important for option pricing:

<div style="display: grid; grid-template-columns: 1fr 1fr; gap: 15px; margin: 20px 0;">
  <div style="background-color: #E57373; color: white; padding: 15px; border-radius: 8px;">
    <h3 style="margin-top: 0;">Correlation (ρ)</h3>
    <p><b>What it is:</b> The correlation between stock returns and changes in implied volatility</p>
    <p><b>Typical values:</b> Usually negative (-0.7 to -0.9 for equities)</p>
    <p><b>Significance:</b> Captures the "leverage effect" - when stock prices fall, volatility tends to rise</p>
  </div>
  
  <div style="background-color: #81C784; color: white; padding: 15px; border-radius: 8px;">
    <h3 style="margin-top: 0;">Hurst Exponent (H)</h3>
    <p><b>What it is:</b> Measures the "roughness" of the volatility process</p>
    <p><b>Values interpretation:</b></p>
    <ul>
      <li>H < 0.5: Rough/anti-persistent process</li>
      <li>H = 0.5: Random walk</li>
      <li>H > 0.5: Smooth/persistent process</li>
    </ul>
    <p><b>Significance:</b> Empirical studies show volatility typically has H ≈ 0.1</p>
  </div>
  
  <div style="background-color: #64B5F6; color: white; padding: 15px; border-radius: 8px;">
    <h3 style="margin-top: 0;">Volatility of Volatility (η)</h3>
    <p><b>What it is:</b> How much the volatility itself fluctuates</p>
    <p><b>Significance:</b> Higher values mean more volatile volatility, leading to fatter tails in return distributions</p>
    <p><b>Impact:</b> Affects the pricing of options, especially those far from at-the-money</p>
  </div>
  
  <div style="background-color: #FFB74D; color: white; padding: 15px; border-radius: 8px;">
    <h3 style="margin-top: 0;">Initial Variance (ξ)</h3>
    <p><b>What it is:</b> The starting point for the variance process</p>
    <p><b>Calculation:</b> Square of the current implied volatility</p>
    <p><b>Importance:</b> Sets the initial level of market volatility in your model</p>
  </div>
</div>

<p style="text-align: center; font-style: italic;">These parameters are crucial inputs for rough volatility models, which can price options more accurately than traditional models.</p>
</div>

<div style="background-color: #4A148C; color: white; padding: 20px; border-radius: 10px; margin-top: 30px; margin-bottom: 20px;">
<h2 style="text-align: center; margin-top: 0; color: white;">🎯 Conclusion</h2>

<p style="font-size: 16px; text-align: center; margin-bottom: 20px;">In this tutorial, we've implemented a complete pipeline for option parameter estimation</p>

<div style="display: flex; justify-content: space-around; flex-wrap: wrap; margin: 20px 0;">
  <div style="background-color: rgba(255,255,255,0.15); border-radius: 8px; padding: 15px; margin: 5px; width: 45%;">
    <h4 style="color: #CE93D8; margin-top: 0;">📊 Data Processing</h4>
    <ul>
      <li>Load and clean financial options data</li>
      <li>Filter by ticker, date, and maturity</li>
      <li>Extract key pricing inputs</li>
    </ul>
  </div>
  <div style="background-color: rgba(255,255,255,0.15); border-radius: 8px; padding: 15px; margin: 5px; width: 45%;">
    <h4 style="color: #CE93D8; margin-top: 0;">📈 Parameter Estimation</h4>
    <ul>
      <li>Correlation (ρ) via returns/volatility analysis</li>
      <li>Roughness (H) via Hurst exponent calculation</li>
      <li>Volatility of volatility (η) and initial variance (ξ)</li>
    </ul>
  </div>
</div>

<p>These parameter estimates can now be used in rough volatility models to:</p>
<ul>
  <li>Calculate theoretical option prices</li>
  <li>Compare with market prices to identify mispricing</li>
  <li>Build trading strategies based on model insights</li>
</ul>

<div style="text-align: center; margin-top: 15px; font-style: italic; color: #CE93D8;">
Next steps: Implement a pricing model using these parameters and calculate option price bounds
</div>
</div>

## 10. Parameter Sensitivity Analysis
<div style="background-color: #E1F5FE; padding: 15px; border-radius: 8px; border-left: 5px solid #03A9F4; margin-bottom: 20px;">
Let's visualize how changes in our key parameters affect option prices. This sensitivity analysis helps us understand which parameters have the largest impact on valuation.

<p style="text-align: center; font-style: italic; margin-top: 10px; color: #0288D1;">Understanding parameter sensitivity is crucial for risk management and model calibration.</p>
</div>

In [13]:
# Sensitivity analysis function
def plot_parameter_sensitivity(results):
    if not isinstance(results, dict):
        print("No valid results to analyze")
        return
    
    # Create a figure with 2x2 subplots for our 4 key parameters
    fig, axs = plt.subplots(2, 2, figsize=(14, 10))
    fig.suptitle(f'Parameter Sensitivity Analysis for {results["ticker"]} Options', 
                fontsize=16, y=0.98)
    
    # 1. Correlation (rho) sensitivity
    ax1 = axs[0, 0]
    rho_values = np.linspace(-1, 0, 100)  # Range of correlation values
    
    # Simulate option price changes with rho (simplified model)
    base_price = results['market_premium']
    # This is a simplified relationship - real pricing would use a full model
    rho_impact = base_price * (1 + 0.2 * (rho_values - results['estimated_rho']))
    
    ax1.plot(rho_values, rho_impact, 'b-', linewidth=2)
    ax1.axvline(x=results['estimated_rho'], color='red', linestyle='--', 
               label=f'Current ρ = {results["estimated_rho"]:.2f}')
    ax1.axhline(y=base_price, color='green', linestyle=':', label=f'Market price = ${base_price:.2f}')
    ax1.set_title('Option Price Sensitivity to Correlation (ρ)', fontsize=12)
    ax1.set_xlabel('Correlation (ρ)')
    ax1.set_ylabel('Estimated Option Price ($)')
    ax1.legend()
    ax1.grid(alpha=0.3)
    
    # 2. Hurst exponent sensitivity
    ax2 = axs[0, 1]
    h_values = np.linspace(0.05, 0.5, 100)  # Range of Hurst exponent values
    
    # Simulate option price changes with H (simplified model)
    # Rough volatility (low H) typically leads to higher implied vol for short maturities
    h_factor = 1 - 2 * (h_values - 0.1)  # Normalized impact factor
    h_impact = base_price * (1 + 0.15 * h_factor)
    
    ax2.plot(h_values, h_impact, 'g-', linewidth=2)
    ax2.axvline(x=results['hurst_exponent'], color='red', linestyle='--', 
               label=f'Current H = {results["hurst_exponent"]:.3f}')
    ax2.axhline(y=base_price, color='green', linestyle=':', label=f'Market price = ${base_price:.2f}')
    ax2.set_title('Option Price Sensitivity to Hurst Exponent (H)', fontsize=12)
    ax2.set_xlabel('Hurst Exponent (H)')
    ax2.set_ylabel('Estimated Option Price ($)')
    ax2.legend()
    ax2.grid(alpha=0.3)
    
    # 3. Volatility of volatility (eta) sensitivity
    ax3 = axs[1, 0]
    eta_base = results['eta']
    eta_values = np.linspace(eta_base * 0.5, eta_base * 1.5, 100)  # Range around current eta
    
    # Higher vol-of-vol increases option prices, especially for OTM options
    moneyness = results['strike_price'] / results['forward_price']  # Put moneyness
    otm_factor = max(0.5, abs(1 - moneyness))  # OTM factor
    
    eta_impact = base_price * (1 + otm_factor * 0.3 * (eta_values/eta_base - 1))
    
    ax3.plot(eta_values, eta_impact, 'purple', linewidth=2)
    ax3.axvline(x=eta_base, color='red', linestyle='--', 
               label=f'Current η = {eta_base:.3f}')
    ax3.axhline(y=base_price, color='green', linestyle=':', label=f'Market price = ${base_price:.2f}')
    ax3.set_title('Option Price Sensitivity to Vol-of-Vol (η)', fontsize=12)
    ax3.set_xlabel('Volatility of Volatility (η)')
    ax3.set_ylabel('Estimated Option Price ($)')
    ax3.legend()
    ax3.grid(alpha=0.3)
    
    # 4. Initial variance (xi) sensitivity
    ax4 = axs[1, 1]
    impl_vol = np.sqrt(results['xi'])  # Back-calculate implied vol
    vol_values = np.linspace(impl_vol * 0.7, impl_vol * 1.3, 100)  # Range of volatility values
    xi_values = vol_values ** 2  # Convert to variance
    
    # Black-Scholes approximation of option price sensitivity to volatility
    # Simplified linear relationship with volatility (in reality more complex)
    vol_impact = base_price * vol_values / impl_vol
    
    ax4.plot(vol_values, vol_impact, 'orange', linewidth=2)
    ax4.axvline(x=impl_vol, color='red', linestyle='--', 
               label=f'Current vol = {impl_vol:.3f}')
    ax4.axhline(y=base_price, color='green', linestyle=':', label=f'Market price = ${base_price:.2f}')
    ax4.set_title('Option Price Sensitivity to Implied Volatility', fontsize=12)
    ax4.set_xlabel('Implied Volatility')
    ax4.set_ylabel('Estimated Option Price ($)')
    ax4.legend()
    ax4.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.92)
    plt.show()
    
    # Create a parameter importance chart
    plt.figure(figsize=(10, 6))
    
    # Calculate the price sensitivity (simplified)
    sensitivities = {
        'Correlation (ρ)': np.max(np.abs(rho_impact - base_price)) / base_price,
        'Hurst Exponent (H)': np.max(np.abs(h_impact - base_price)) / base_price,
        'Volatility of Vol (η)': np.max(np.abs(eta_impact - base_price)) / base_price,
        'Implied Volatility': np.max(np.abs(vol_impact - base_price)) / base_price
    }
    
    # Sort by importance
    sorted_sens = dict(sorted(sensitivities.items(), key=lambda x: x[1], reverse=True))
    
    # Plot as horizontal bar chart
    colors = ['#3498db', '#2ecc71', '#9b59b6', '#e74c3c']
    plt.barh(list(sorted_sens.keys()), list(sorted_sens.values()), color=colors, alpha=0.7)
    
    plt.title('Parameter Sensitivity Ranking', fontsize=14)
    plt.xlabel('Relative Impact on Option Price', fontsize=12)
    
    # Add percentage labels
    for i, (param, value) in enumerate(sorted_sens.items()):
        plt.text(value + 0.01, i, f'{value*100:.1f}%', va='center', fontweight='bold')
    
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.show()

# Run the sensitivity analysis on our results
if 'results' in locals() and isinstance(results, dict):
    plot_parameter_sensitivity(results)
else:
    print("Please run the analysis first to generate results")

Please run the analysis first to generate results


## 11. Next Steps
<div style="background-color: #FFF3E0; padding: 15px; border-radius: 8px; border-left: 5px solid #FF9800; margin-top: 20px;">
<h3 style="margin-top: 0; color: #E65100;">Further Analysis and Applications</h3>

<p>Now that we've estimated our model parameters and analyzed their sensitivity, here are some ways to extend this analysis:</p>

<ol style="color: #333;">
  <li><b>Implement a full rough volatility pricer</b> using the estimated parameters</li>
  <li><b>Compare model prices to market prices</b> across different strikes and maturities</li>
  <li><b>Conduct backtesting</b> to validate parameter stability over time</li>
  <li><b>Develop trading strategies</b> based on model/market price discrepancies</li>
  <li><b>Explore machine learning approaches</b> to parameter calibration</li>
</ol>

<p style="text-align: center; margin-top: 15px; font-style: italic; color: #E65100;">
The integration of rough volatility models can lead to more accurate option pricing and improved risk management.
</p>
</div>